In [1]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku 
import numpy as np

Using TensorFlow backend.


In [3]:
# data = open("andy.txt",encoding="utf8").read().replace('\n',' ')
data = open("NEU/data/andy.txt",encoding="utf8").read()
data = data.replace("'","")
data

'Rita Hayworth and the Shawshank Redemption\n\nTheres a guy like me in every state and federal prison in America, I guess--Im\nthe guy who can get it for you. Tailor made cigarettes, a bag of reefer if \nyoure\npartial to that, a bottle of brandy to celebrate your son or daughters high\n\nschool graduation, or anything else within reason, that is.  It wasnt always \nthat way.\n\nI came to Shawshank when I was just twenty, and I am one of the few people in\nour happy little family willing to own up to what he did.  I committed murder.  \nI\nput a large insurance policy on my wife -- who was three years older than I was \n--\nand then I fixed the brakes on the Chevrolet coupe her father had given us as a\nwedding present.  It worked out exactly as I had planned -- except I hadnt \nplanned\n\non her stopping to pick up the neighbor woman and the neighbor woman’s infant \nson\non their way down Castle Hill and into town.  The brakes let go and the car\n\ncrashed through the bushes at the e

In [4]:
tokenizer = Tokenizer()
def dataset_preparation(data):
    corpus = data.lower().split("\n")
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
    predictors, label = input_sequences[:,:-1], input_sequences[:,-1]
    label = ku.to_categorical(label, num_classes=total_words)
    return predictors,label,max_sequence_len,total_words

In [5]:
def create_model(predictors, label, max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    model.add(Embedding(total_words, 10, input_length=input_len))
    model.add(LSTM(150))
    model.add(Dropout(0.1))
    model.add(Dense(total_words, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    model.fit(predictors, label, epochs=100, verbose=1)
    return model

In [6]:
def generate_text(seed_text, next_words, max_sequence_len, model):
    for j in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen= 
                             max_sequence_len-1, padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)
  
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

In [7]:
X, Y, max_len, total_words = dataset_preparation(data)
model = create_model(X, Y, max_len, total_words)

C:\Users\Karti\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/100
37392/37392 [==============================] - 27s 730us/step - loss: 6.5804
Epoch 2/100
37392/37392 [==============================] - 26s 703us/step - loss: 6.2652
Epoch 3/100
37392/37392 [==============================] - 28s 746us/step - loss: 6.0914
Epoch 4/100
37392/37392 [==============================] - 28s 758us/step - loss: 5.9357
Epoch 5/100
37392/37392 [==============================] - 28s 749us/step - loss: 5.7689
Epoch 6/100
37392/37392 [==============================] - 29s 778us/step - loss: 5.5933
Epoch 7/100
37392/37392 [==============================] - 29s 775us/step - loss: 5.4078
Epoch 8/100
37392/37392 [==============================] - 29s 766us/step - loss: 5.2155
Epoch 9/100
37392/37392 [==============================] - 28s 757us/step - loss: 5.0254
Epoch 10/100
37392/37392 [==============================] - 28s 756us/step - loss: 4.8360
Epoch 11/100
37392/37392 [==============================] - 28s 758us/step - loss: 4.6508
Epoch 12/100
37392/

In [24]:
text = generate_text("Andy was", 10, max_len, model)
print(text)

Andy was silent for a moment that was like that alone i


In [27]:
text = generate_text("Samuel did", 10, max_len, model)
print(text)

Samuel did it because he had gotten a high school diploma as


In [31]:
text = generate_text("Hadley was", 16, max_len, model)
print(text)

Hadley was now looking at the country club that part of it he was an real out of


In [34]:
text = generate_text("The Sisters gang", 16, max_len, model)
print(text)

The Sisters gang right from the infirmary as the red is to beyond his end of the country back


In [35]:
text = generate_text("Tommy was", 16, max_len, model)
print(text)

Tommy was the day on the lock in the town of the time he took his new rock


In [41]:
text = generate_text(" Andy had cigarettes", 10, max_len, model)
print(text)

 Andy had cigarettes reported missing on the outside i have seen the gun


In [43]:
text = generate_text("Brooks was here", 16, max_len, model)
print(text)

Brooks was here there was a lot of brutality at the prison number i so business the sun was


In [44]:
text = generate_text("Ellis is", 16, max_len, model)
print(text)

Ellis is on the american side of the border directly across from el porvenir the prison side of


In [47]:
text = generate_text("How was Andy as a person?", 16, max_len, model)
print(text)

How was Andy as a person? and even even that and seemed to tell him the boston number to be how it


In [48]:
text = generate_text("Is Tommy alive?", 16, max_len, model)
print(text)

Is Tommy alive? took him the boston register cellblock rock died of their hour and the maine had by


In [50]:
text = generate_text("There's a guy like me in every state and federal prison in America", 8, max_len, model)
print(text)

There's a guy like me in every state and federal prison in America i guess im a man like this rock


In [51]:
text = generate_text("There's a guy like me in every state and federal prison in America i guess im a man like this rock", 8, max_len, model)
print(text)

There's a guy like me in every state and federal prison in America i guess im a man like this rock hammer the year but andy had left me


In [53]:
text = generate_text("Rita Hayworth and the Shawshank Redemption", 10000, max_len, model)
print(text)

Rita Hayworth and the Shawshank Redemption right out of the infirmary and the master who had been out of the far club of the month nice like my mind is as the rock red outer other three years in all at his wife 30 found a birth before that night he can young way andy turns over my head into the river no i weak the night there or a big stock man at a small pipe like country by the kid on the next if he put him we his friend had a friend man in the infirmary to some five worth minutes i only now in the wall the rest of camels to keep your new a little man with the window of the end of his mind if he done a few just from my kid clear that the prison woman by his time before i did is at that much as ive been taken to him if dont remember to be him strong woman as andy thought it was wed be for gratitude in the nine or dutch wife like posters ive could be to sixteen to the so track i might have how the long or six or the fisherman time to me about a man at the time if it is i guess for how 

# THE END!
## Finally we've got something we can show off in our Data Science skills :)

<br />
<center>
<img src =NEU/ipynb.images/shawshank-5.jpg width = 600 />
</center>
